In [2]:
from rasterstats import zonal_stats
import geopandas as gpd

import glob
import os
import numpy as np

from configs.config_object import ConfigObject

In [3]:
config = ConfigObject('../configs/config.json')

## Load bathymetry asc files

In [4]:
# List of ASC files
path_swisstopo = config.paths.swiss_topo_path
path_grid = config.paths.grid_folder_path

## Load MITgcm grid

In [6]:
grid = gpd.read_file(os.path.join(path_grid, "grid_epsg2056.gpkg"))

In [38]:
# to get the grid shape
with open(os.path.join(path_grid, 'x_sg_grid.npy'), 'rb') as f:
    x = np.load(f)

## Compute average depth per cell

In [24]:
# Compute zonal statistics
stats = zonal_stats(grid, os.path.join(path_swisstopo, "merged_epsg2056.tif"), stats="mean", nodata=0)

In [30]:
grid["avg_altitude"] = [s["mean"] for s in stats]

In [31]:
grid["avg_altitude"].max()

371.66381278538813

In [32]:
grid["avg_depth"] = grid["avg_altitude"] - config.lake_altitude

In [36]:
grid["avg_depth"].head()

0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
Name: avg_depth, dtype: float64

In [35]:
grid["avg_depth"] = grid["avg_depth"].fillna(2)
depth_array = grid["avg_depth"].to_numpy().reshape(x.shape)

## Save bathymetry

In [37]:
# Write gpkg file
grid.to_file(os.path.join(config.paths.grid_folder_path, "grid_with_depth_epsg2056.gpkg"), driver="GPKG")

In [42]:
# Write binary file
fid = open(os.path.join(config.paths.grid_folder_path, "bathy.bin"), 'wb')
depth_array.astype('>f8').tofile(fid)
fid.close()